In [36]:
import sys
sys.path.append('/Users/tompease/Documents/Coding/airbnb')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
import utils.data_loader as utils
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

loader = utils.AirbnbLoader()
X, y = loader.load_airbnb('Category', normalized=True)

encoder = LabelEncoder()
y = encoder.fit_transform(y)

cv_split = ShuffleSplit(n_splits = 5, test_size = .3, train_size = .7, random_state = 42)

In [45]:
classification_algs = [
  LogisticRegression(max_iter=1000),
  RandomForestClassifier(),
  GradientBoostingClassifier(),
  SVC(),
  GaussianNB(),
  MultinomialNB(),
  SGDClassifier(),
  KNeighborsClassifier(),
  DecisionTreeClassifier(),
  XGBClassifier()
]

columns = ['MLA name', 'Parameters', 'Train accuracy', 'Test accuracy', 'Train f1 weighted', 'Test f1 weighted']
MLA_compare = pd.DataFrame(columns = columns)

row_index = 0

for alg in classification_algs:
  name = alg.__class__.__name__
  MLA_compare.loc[row_index, 'MLA name'] = name
  MLA_compare.loc[row_index, 'Parameters'] = str(alg.get_params())

  cv_results = cross_validate(alg, X, y, scoring=['f1_weighted', 'accuracy'], return_train_score=True)

  MLA_compare.loc[row_index, 'Train accuracy'] = cv_results['train_accuracy'].mean()
  MLA_compare.loc[row_index, 'Test accuracy'] = cv_results['test_accuracy'].mean()
  MLA_compare.loc[row_index, 'Train f1 weighted'] = cv_results['train_f1_weighted'].mean()
  MLA_compare.loc[row_index, 'Test f1 weighted'] = cv_results['test_f1_weighted'].mean()

  row_index += 1

MLA_compare.sort_values(by = ['Test accuracy'], ascending=False, inplace = True)

MLA_compare

,MLA name,Parameters,Train accuracy,Test accuracy,Train f1 weighted,Test f1 weighted
0,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False...",0.395181,0.36988,0.369506,0.341468
2,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.833735,0.36506,0.835006,0.359094
9,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...",0.996386,0.343373,0.996386,0.339606
1,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.996386,0.343373,0.996385,0.336756
4,GaussianNB,"{'priors': None, 'var_smoothing': 1e-09}",0.335241,0.321687,0.27115,0.251464
5,MultinomialNB,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.319277,0.315663,0.257528,0.249679
3,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.30512,0.304819,0.238304,0.236285
8,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...",0.996386,0.298795,0.996387,0.298287
7,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.503916,0.279518,0.491028,0.257371
6,SGDClassifier,"{'alpha': 0.0001, 'average': False, 'class_wei...",0.237651,0.23012,0.144248,0.133218
